LIB CREDITS 1:
https://twitter.com/abhi1thakur

CODE & DATASET CREDITS 2: https://twitter.com/@josh_bickett  

In [ ]:
!nvidia-smi

Fri Sep 29 20:16:59 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Fine-tuning Mistral 7b with AutoTrain

In this notebook, we'll walk you through the steps to fine-tune Mistral 7b using your dataset.
Follow along by running each cell in order!

### Setup Runtime
For fine-tuning Llama, a GPU instance is essential. Follow the directions below:

1. Go to `Runtime` (located in the top menu bar).
2. Select `Change Runtime Type`.
3. Choose `T4 GPU` (or a comparable option).


### Package Installation

Before we get started, let's ensure we have all the necessary packages installed.

In [ ]:
!pip install -q pandas


In [ ]:
!pip install -q autotrain-advanced


### Setup Autotrain
The step below is required for AutoTrain in Colab


In [ ]:
!autotrain setup --update-torch


> INFO    Installing latest transformers@main
> INFO    Successfully installed latest transformers
> INFO    Installing latest peft@main
> INFO    Successfully installed latest peft
> INFO    Installing latest diffusers@main
> INFO    Successfully installed latest diffusers
> INFO    Installing latest trl@main
> INFO    Successfully installed latest trl
> INFO    Installing latest xformers
> INFO    Successfully installed latest xformers
> INFO    Installing latest PyTorch
> INFO    Successfully installed latest PyTorch


### Connect to Hugging Face for model upload
#### Logging to Hugging Face

To make sure the model can be uploaded to be used for Inference, it's necessary to log in to the Hugging Face hub.

#### Getting a Hugging Face token
Steps:
1. Navigate to this URL: https://huggingface.co/settings/tokens
2. Create a `write` token and copy it to your clipboard
3. Run the code below and enter your token



In [ ]:
from huggingface_hub import notebook_login
notebook_login()

### Upload your dataset
Add your data set to the root directory in the Colab under the name `train.csv`. The AutoTrain command will look for your data there under that name.

##### Don't have a data set and want to try finetuning on an example data set?
If you don't have a dataset you can run these commands below to get an example data set and save it to `train.csv`






In [ ]:
!git clone https://github.com/joshbickett/finetune-llama-2.git
%cd finetune-llama-2
%mv train.csv ../train.csv
%cd ..

In [ ]:
import pandas as pd
df = pd.read_csv("train.csv")
df

,Concept,Funny Description Prompt,text
0,A person walks in the rain,"A person walks in the rain, wearing a suit mad...",###Human:\ngenerate a midjourney prompt for A ...
1,A cat chasing a mouse,"A cat, wearing detective attire and sunglasses...",###Human:\ngenerate a midjourney prompt for A ...
2,A dog eating a bone,A dog with a chef's hat is eating a bone seaso...,###Human:\ngenerate a midjourney prompt for A ...
3,A bird flying in the sky,A bird with oversized sunglasses and a basebal...,###Human:\ngenerate a midjourney prompt for A ...
4,A fish swimming in a tank,A fish in a snorkel and flippers swimming in a...,###Human:\ngenerate a midjourney prompt for A ...
5,A child playing with toys,A child in a superhero cape playing with alien...,###Human:\ngenerate a midjourney prompt for A ...
6,A car driving on the road,A car with legs instead of wheels running down...,###Human:\ngenerate a midjourney prompt for A ...
7,A flower blooming in a garden,A flower wearing a top hat and bow tie bloomin...,###Human:\ngenerate a midjourney prompt for A ...
8,A bee collecting pollen,A bee with a backpack vacuum cleaner collectin...,###Human:\ngenerate a midjourney prompt for A ...
9,A sun setting over the ocean,The sun wearing sunglasses and sipping on a tr...,###Human:\ngenerate a midjourney prompt for A ...


In [ ]:
df['text'][13]

'###Human:\ngenerate a midjourney prompt for A lion roaring in the jungle\n\n###Assistant:\nA lion with a microphone, singing a ballad in the jungle with a group of birds as backup singers.'

## Overview of AutoTrain command

#### Short overview of what the command flags do.

- `!autotrain`: Command executed in environments like a Jupyter notebook to run shell commands directly. `autotrain` is an automatic training utility.

- `llm`: A sub-command or argument specifying the type of task

- `--train`: Initiates the training process.

- `--project_name`: Sets the name of the project

- `--model abhishek/llama-2-7b-hf-small-shards`: Specifies original model that is hosted on Hugging Face named "llama-2-7b-hf-small-shards" under the "abhishek".

- `--data_path .`: The path to the dataset for training. The "." refers to the current directory. The `train.csv` file needs to be located in this directory.

- `--use_int4`: Use of INT4 quantization to reduce model size and speed up inference times at the cost of some precision.

- `--learning_rate 2e-4`: Sets the learning rate for training to 0.0002.

- `--train_batch_size 12`: Sets the batch size for training to 12.

- `--num_train_epochs 3`: The training process will iterate over the dataset 3 times.

### Steps needed before running
Go to the `!autotrain` code cell below and update it by following the steps below:

1. After `--project_name` replace `*enter-a-project-name*` with the name that you'd like to call the project
2. After `--repo_id` replace `*username*/*repository*`. Replace `*username*` with your Hugging Face username and `*repository*` with the repository name you'd like it to be created under. You don't need to create this repository before hand, it will automatically be created and uploaded once the training is completed.
3. Confirm that `train.csv` is in the root directory in the Colab. The `--data_path .` flag will make it so that AutoTrain looks for your data there.
4. Make sure to add the LoRA Target Modules to be trained `--target-modules q_proj, v_proj`
5. Once you've made these changes you're all set, run the command below!




In [ ]:
!autotrain llm --train --project_name mistral-7b-mj-finetuned --model bn22/Mistral-7B-Instruct-v0.1-sharded --data_path . --use_peft --use_int4 --learning_rate 2e-4 --train_batch_size 12 --num_train_epochs 3 --trainer sft --target_modules q_proj,v_proj --push_to_hub --repo_id 1littlecoder/mistral-7b-mj-finetuned

> INFO    Running LLM
> INFO    Params: Namespace(version=False, train=True, deploy=False, inference=False, data_path='.', train_split='train', valid_split=None, text_column='text', model='bn22/Mistral-7B-Instruct-v0.1-sharded', learning_rate=0.0002, num_train_epochs=3, train_batch_size=12, warmup_ratio=0.1, gradient_accumulation_steps=1, optimizer='adamw_torch', scheduler='linear', weight_decay=0.0, max_grad_norm=1.0, seed=42, add_eos_token=False, block_size=-1, use_peft=True, lora_r=16, lora_alpha=32, lora_dropout=0.05, logging_steps=-1, project_name='mistral-7b-mj-finetuned', evaluation_strategy='epoch', save_total_limit=1, save_strategy='epoch', auto_find_batch_size=False, fp16=False, push_to_hub=True, use_int8=False, model_max_length=1024, repo_id='1littlecoder/mistral-7b-mj-finetuned', use_int4=True, trainer='sft', target_modules='q_proj,v_proj', merge_adapter=False, token=None, backend='default', username=None, use_flash_attention_2=False, func=<function run_llm_command_factory 

## Completed 🎉
After the command above is completed your Model will be uploaded to Hugging Face.

#### Learn more about AutoTrain (optional)
If you want to learn more about what command-line flags are available


In [ ]:

!autotrain llm -h

In [ ]:
!pip install -q safetensors

In [ ]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "bn22/Mistral-7B-Instruct-v0.1-sharded"
#adapters_name = "1littlecoder/mistral-7b-mj-finetune"

device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    model_name,
   # load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    device_map='auto'
)
##model = PeftModel.from_pretrained(model, adapters_name)
#model = model.merge_and_unload()
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.bos_token_id = 1

stop_token_ids = [0]

print(f"Successfully loaded the model {model_name} into memory")

KeyboardInterrupt: ignored

In [ ]:

text = "<s>[INST] What is your favourite condiment? [/INST]"
"Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s> "
"[INST] Do you have mayonnaise recipes? [/INST]"

encodeds = tokenizer(text, return_tensors="pt", add_special_tokens=False)

model_inputs = encodeds.to(device)
model.to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])